In [1]:
# Dependências
import requests
import zipfile
import io
import pandas as pd
import unicodedata
import re

#### Dados Votação

In [2]:
# Download do arquivo ZIP em memória
response = requests.get("https://cdn.tse.jus.br/estatistica/sead/eleicoes/eleicoes2022/correspefet/CEFT_2t_RN_311020221100.zip" )
if response.status_code == 200:
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        csv_file = [file for file in z.namelist() if file.endswith(".csv")][0]
        with z.open(csv_file) as f:
            df_votos = pd.read_csv(f, encoding="ISO-8859-1", sep=";")
else:
    print(f"Falha ao baixar o arquivo. Código de status: {response.status_code}")


In [3]:
df_votos.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_PLEITO,DT_PLEITO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,...,CD_CARGA_URNA_EFETIVADA,CD_CARGA_1_URNA_EFETIVADA,CD_CARGA_2_URNA_EFETIVADA,CD_FLASHCARD_URNA_EFETIVADA,DT_CARGA_URNA_EFETIVADA,DT_RECO_URNA_EFETIVADA,CD_ORIGEM_VOTO,DS_ORIGEM_VOTO,DS_DIVERGENCIA,NR_LOCAL_VOTACAO
0,31/10/2022,11:01:03,2022,407,30/10/2022,RN,16004,TENENTE LAURENTINO CRUZ,21,16,...,444569170192552795523074,444.569.170.192.552.795.,523.074,931D48EE,20/09/2022 18:14,20/09/2022 18:29,U,Urna Eletrônica,#NULO#,1015
1,31/10/2022,11:01:03,2022,407,30/10/2022,RN,16004,TENENTE LAURENTINO CRUZ,21,17,...,181943028166273422629247,181.943.028.166.273.422.,629.247,931D48EE,20/09/2022 16:43,20/09/2022 18:29,U,Urna Eletrônica,#NULO#,1015
2,31/10/2022,11:01:03,2022,407,30/10/2022,RN,16004,TENENTE LAURENTINO CRUZ,21,18,...,524777525572328425459658,524.777.525.572.328.425.,459.658,BC806D0A,21/09/2022 10:47,21/09/2022 11:01,U,Urna Eletrônica,#NULO#,1015
3,31/10/2022,11:01:03,2022,407,30/10/2022,RN,16004,TENENTE LAURENTINO CRUZ,21,44,...,596084319607911326086789,596.084.319.607.911.326.,86.789,931D48EE,20/09/2022 16:55,20/09/2022 18:29,U,Urna Eletrônica,#NULO#,1015
4,31/10/2022,11:01:03,2022,407,30/10/2022,RN,16004,TENENTE LAURENTINO CRUZ,21,47,...,353660402883987003028961,353.660.402.883.987.003.,28.961,931D48EE,20/09/2022 17:00,20/09/2022 18:29,U,Urna Eletrônica,#NULO#,1015


#### Dados Censitários - Por Distritos

In [4]:
'''
DPOs = Domicílios Particulares Ocupados
'''

df_distritos = pd.read_csv('Bases/Distritos.csv').rename(columns={'v0001': 'Pessoas',
                                                                  'v0002': 'Domicílios',
                                                                  'v0003': 'Domicílios Particulares',
                                                                  'v0004': 'Domicílios Coletivos',
                                                                  'v0005': 'Média de moradores em DPOs',
                                                                  'v0006': 'Percentual de DPOs Imputados',
                                                                  'v0007': 'Domicílios Particulares Ocupados'
                                                                  })
df_distritos.head()

,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,CD_DIST,NM_MUN,NM_DIST,Total_setores,Pessoas,Domicílios,Domicílios Particulares,Domicílios Coletivos,Média de moradores em DPOs,Percentual de DPOs Imputados,Domicílios Particulares Ocupados
0,2,Nordeste,24,Rio Grande do Norte,2400109,240010905,Acari,Acari,28,10597,4750,4747,3,2.768747,0.996329,3814
1,2,Nordeste,24,Rio Grande do Norte,2400208,240020805,Açu,Açu,104,56496,24138,24103,35,2.932901,1.890418,19255
2,2,Nordeste,24,Rio Grande do Norte,2400307,240030705,Afonso Bezerra,Afonso Bezerra,25,10839,4339,4334,5,3.211385,0.177883,3373
3,2,Nordeste,24,Rio Grande do Norte,2400406,240040605,Água Nova,Água Nova,4,2946,1199,1199,0,2.972755,0.201816,991
4,2,Nordeste,24,Rio Grande do Norte,2400505,240050505,Alexandria,Alexandria,22,13640,6043,6040,3,2.811687,0.392319,4843


In [5]:
df_renda = pd.read_csv('Bases/Rendimento.csv').rename(columns={'Total': 'Total Indivíduos'})
df_renda.head()

,Distrito,Total Indivíduos,Até 1/2,Mais de 1/2 a 1,Mais de 1 a 2,Mais de 2 a 5,Mais de 5 a 10,Mais de 10 a 20,Mais de 20,Sem rendimento,Código da Unidade Geográfica
0,Total,2675748,288918,717278,323928,175559,60684,22246,7715,1078875,24
1,Acari,9537,1024,3101,1001,478,115,31,7,3779,2400109
2,Açu,44769,5554,13101,4715,2331,625,167,53,18208,2400208
3,Afonso Bezerra,8927,1789,2442,656,211,24,11,3,3791,2400307
4,Água Nova,2533,333,662,160,63,6,0,0,1309,2400406


In [6]:
df_censitario = pd.merge(df_distritos, df_renda, left_on='NM_DIST', right_on='Distrito', how='inner')
df_censitario.head()

,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,CD_DIST,NM_MUN,NM_DIST,Total_setores,Pessoas,...,Total Indivíduos,Até 1/2,Mais de 1/2 a 1,Mais de 1 a 2,Mais de 2 a 5,Mais de 5 a 10,Mais de 10 a 20,Mais de 20,Sem rendimento,Código da Unidade Geográfica
0,2,Nordeste,24,Rio Grande do Norte,2400109,240010905,Acari,Acari,28,10597,...,9537,1024,3101,1001,478,115,31,7,3779,2400109
1,2,Nordeste,24,Rio Grande do Norte,2400208,240020805,Açu,Açu,104,56496,...,44769,5554,13101,4715,2331,625,167,53,18208,2400208
2,2,Nordeste,24,Rio Grande do Norte,2400307,240030705,Afonso Bezerra,Afonso Bezerra,25,10839,...,8927,1789,2442,656,211,24,11,3,3791,2400307
3,2,Nordeste,24,Rio Grande do Norte,2400406,240040605,Água Nova,Água Nova,4,2946,...,2533,333,662,160,63,6,0,0,1309,2400406
4,2,Nordeste,24,Rio Grande do Norte,2400505,240050505,Alexandria,Alexandria,22,13640,...,11487,2191,3412,807,363,78,16,6,4614,2400505


In [7]:
'''
Normalizando os nomes dos distritos para eles baterem com df_votos
'''

def normalize_text(value):
    if isinstance(value, str):
        # Removendo acentos e caracteres especiais
        value = ''.join(
            char for char in unicodedata.normalize('NFD', value)
            if unicodedata.category(char) != 'Mn'
        )
        # Convertendo para letra maiúscula
        value = value.upper()
        # Removendo caracteres não-alfanuméricos exceto espaços
        value = re.sub(r'[^A-Z0-9 ]', '', value)
    return value

# Aplicando a função nas colunas
df_censitario['NM_MUN'] = df_censitario['NM_MUN'].apply(normalize_text)
df_censitario['NM_DIST'] = df_censitario['NM_DIST'].apply(normalize_text)
df_censitario.head()

,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,CD_DIST,NM_MUN,NM_DIST,Total_setores,Pessoas,...,Total Indivíduos,Até 1/2,Mais de 1/2 a 1,Mais de 1 a 2,Mais de 2 a 5,Mais de 5 a 10,Mais de 10 a 20,Mais de 20,Sem rendimento,Código da Unidade Geográfica
0,2,Nordeste,24,Rio Grande do Norte,2400109,240010905,ACARI,ACARI,28,10597,...,9537,1024,3101,1001,478,115,31,7,3779,2400109
1,2,Nordeste,24,Rio Grande do Norte,2400208,240020805,ACU,ACU,104,56496,...,44769,5554,13101,4715,2331,625,167,53,18208,2400208
2,2,Nordeste,24,Rio Grande do Norte,2400307,240030705,AFONSO BEZERRA,AFONSO BEZERRA,25,10839,...,8927,1789,2442,656,211,24,11,3,3791,2400307
3,2,Nordeste,24,Rio Grande do Norte,2400406,240040605,AGUA NOVA,AGUA NOVA,4,2946,...,2533,333,662,160,63,6,0,0,1309,2400406
4,2,Nordeste,24,Rio Grande do Norte,2400505,240050505,ALEXANDRIA,ALEXANDRIA,22,13640,...,11487,2191,3412,807,363,78,16,6,4614,2400505


#### Dados Unificados

In [8]:
df_unificado = pd.merge(df_votos, df_censitario, left_on='NM_MUNICIPIO', right_on='NM_DIST', how='inner')
df_unificado.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_PLEITO,DT_PLEITO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,...,Total Indivíduos,Até 1/2,Mais de 1/2 a 1,Mais de 1 a 2,Mais de 2 a 5,Mais de 5 a 10,Mais de 10 a 20,Mais de 20,Sem rendimento,Código da Unidade Geográfica
0,31/10/2022,11:01:03,2022,407,30/10/2022,RN,16004,TENENTE LAURENTINO CRUZ,21,16,...,4293,1105,1159,375,100,19,4,0,1531,2414159
1,31/10/2022,11:01:03,2022,407,30/10/2022,RN,16004,TENENTE LAURENTINO CRUZ,21,17,...,4293,1105,1159,375,100,19,4,0,1531,2414159
2,31/10/2022,11:01:03,2022,407,30/10/2022,RN,16004,TENENTE LAURENTINO CRUZ,21,18,...,4293,1105,1159,375,100,19,4,0,1531,2414159
3,31/10/2022,11:01:03,2022,407,30/10/2022,RN,16004,TENENTE LAURENTINO CRUZ,21,44,...,4293,1105,1159,375,100,19,4,0,1531,2414159
4,31/10/2022,11:01:03,2022,407,30/10/2022,RN,16004,TENENTE LAURENTINO CRUZ,21,47,...,4293,1105,1159,375,100,19,4,0,1531,2414159
